In [1]:
import base64
import os
from PIL import Image
from io import BytesIO
import requests
import time

In [2]:
# authentication
engine_id = "stable-diffusion-v1-6"
api_host = os.getenv('API_HOST', 'https://api.stability.ai')
api_key = "sk-dBeXPNuA8GtHabuncaZ6a7uNHnJlnaLVuaz4olhxXOf3pcOn"

In [3]:
#text to image
def text_to_image(api_key, prompt, debug=False):
    """ generates image based on text prompt and returns image data """
    try:
        response = requests.post(f"{api_host}/v1/generation/{engine_id}/text-to-image",
                                 headers={
                                     "Content-Type": "application/json",
                                     "Accept": "application/json",
                                     "Authorization": f"Bearer {api_key}"},
                                json={
                                    "text_prompts": [
                                        {"text": prompt}
                                    ],
                                    "cfg_scale": 7,
                                    "height": 576,
                                    "width": 1024,
                                    "samples": 1,
                                    "steps": 30,
                                })
        
        data = response.json()
    
        # set debug=true if debugging, will save file to local
        if debug:
            for i, image in enumerate(data["artifacts"]):
                with open(f"./txt2img_{i}.png", "wb") as f:
                    f.write(base64.b64decode(image["base64"]))
        
        image_base64 = response.json()["artifacts"][0]["base64"]  # Assuming only one image is generated
        decoded_image = base64.b64decode(image_base64)
        print(decoded_image)
        return decoded_image

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [16]:
# image to video
def image_to_video(api_key, img, debug=False):
    """ generates video based on image and returns video data """
    try:
        get_response = requests.post(
            f"https://api.stability.ai/v2alpha/generation/image-to-video",
            headers={"authorization": api_key},
            files={"image": img},
            data={
                "seed": 0,
                "cfg_scale": 1.8,
                "motion_bucket_id": 127
            },
        )
        
        get_response.raise_for_status()
        generation_id = get_response.json().get('id')
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during image-to-video generation: {e}")

    try:
        status = 0
        time.sleep(10)
        while status!=200:
            print('tried')
            time.sleep(5)
            post_response = requests.request(
            "GET",
            f"https://api.stability.ai/v2alpha/generation/image-to-video/result/{generation_id}",
            headers={
                'Accept': "video/*",  
                'authorization': api_key
            },)
            post_response.raise_for_status()
            status = post_response.status_code
        print('video Generated')
        # set debug=true if debugging, will save file to local
        if debug:
            with open("video.mp4", 'wb') as file:
                file.write(post_response.content)
                
        return post_response.content
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during image-to-video result retrieval: {e}")

In [9]:
image_data = text_to_image(api_key, "A High resolution loading screen for an dungeons and dragons game",debug=True)

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\x00\x00\x00\x02@\x08\x02\x00\x00\x00\x9c\xf7\x83\x82\x00\x02\x01\xbacaBX\x00\x02\x01\xbajumb\x00\x00\x00\x1ejumdc2pa\x00\x11\x00\x10\x80\x00\x00\xaa\x008\x9bq\x03c2pa\x00\x00\x02\x01\x94jumb\x00\x00\x00Gjumdc2ma\x00\x11\x00\x10\x80\x00\x00\xaa\x008\x9bq\x03urn:uuid:a05d8536-58b1-475f-912e-9e640edd8080\x00\x00\x01\xc2\xecjumb\x00\x00\x00)jumdc2as\x00\x11\x00\x10\x80\x00\x00\xaa\x008\x9bq\x03c2pa.assertions\x00\x00\x01\xc0\x8ejumb\x00\x00\x003jumd@\xcb\x0c2\xbb\x8aH\x9d\xa7\x0b*\xd6\xf4\x7fCi\x03c2pa.thumbnail.claim.jpeg\x00\x00\x00\x00\x14bfdb\x00image/jpeg\x00\x00\x01\xc0?bidb\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xc0\x00\x11\x08\x02@\x04\x00\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x0

In [17]:
video_data = image_to_video(api_key, image_data)

tried
tried
tried
tried
tried
tried
tried
tried
tried
tried
video Generated
